In [5]:
import oracledb
import pandas as pd

In [9]:
username = "SYSTEM"
password = "123"
dsn = "localhost/XE"
connection = oracledb.connect(
    user=username,
    password = password,
    dsn = dsn,
    # mode = oracledb.SYSDBA

)
cursor = connection.cursor()

In [13]:
df_boa = pd.read_csv('../data/BOA_cleaned.csv')
df_cbe = pd.read_csv('../data/CBE_cleaned.csv')
df_dash = pd.read_csv('../data/Dashen_cleaned.csv')
df = pd.concat([df_boa,df_cbe,df_dash])
df["date"] = pd.to_datetime(df["date"], errors="coerce")

In [11]:
bank_data = [
    {"name": "CBE", "website": "https://combanketh.et/"},
    {"name": "BOA", "website": "https://www.bankofabyssinia.com/"},
    {"name": "Dashen Bank", "website": "https://dashenbanksc.com/"},
]

# Convert to DataFrame
banks_df = pd.DataFrame(bank_data)

# Insert banks
for _, row in banks_df.iterrows():
    cursor.execute("""
        INSERT INTO banks (name, website)
        VALUES (:1, :2)
    """, (row["name"], row["website"]))


In [14]:
cursor.execute("SELECT bank_id, name FROM banks")
bank_name_to_id = {name: bank_id for bank_id, name in cursor.fetchall()}

for _, row in df.iterrows():
    bank_name = row["bank_name"]
    
    # Get bank_id, skip if bank not found in DB
    bank_id = bank_name_to_id.get(bank_name)
    if bank_id is None:
        print(f"Warning: Bank '{bank_name}' not found in DB. Skipping review.")
        continue
    
    cursor.execute("""
        INSERT INTO reviews (bank_id, review_text, rating, review_date)
        VALUES (:1, :2, :3, :4)
    """, (
        bank_id,
        row["review_text"],
        float(row["rating"]) if pd.notnull(row["rating"]) else None,
        row["date"].to_pydatetime() if pd.notnull(row["date"]) else None
    ))

In [ ]:
# Commit and close
connection.commit()
cursor.close()
connection.close()

print("✅ Data inserted successfully.")